In [1]:
# A ce stade, nous avons 47 variables. 
# Objectif de ce notebook : sélectionner les variables qui seront utilisées pour la modélisation.
# Nous allons utiliser ici 3 méthodes : 
# - Méthode Wrapper : RFE : Recursive Feature Elimination
# - Sélection des variables par SelectFromModel
# - Sélection des variables par SELECTKBEST - score_func=f_classif

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

In [3]:
df=pd.read_csv("df.csv")

In [4]:
df.head()

,Revente >1,libnatmut,valeurfonc,nblot,l_codinsee,nblocdep,Type appart,scarrez,anarnc202012_nb_log,anarnc202012_nb_lot_garpark,...,baie_orientation_autre,Commerces_bruyants,qty_of_mut,datemut,anneemut,moismut,etaban_lot,cat_valeur,surface_appart,cat_scarrez
0,0,Vente,370000.0,1,75101,1,2P,26.06,158.0,0.0,...,0,1,1,2021-06-04,2021,6,75101_4901_00019_93,Entre 320 & 436k€ €,32,<35 m²
1,0,Vente,451000.0,1,75117,0,2P,35.53,32.0,0.0,...,0,0,1,2021-03-18,2021,3,75117_5503_00103_24,Entre 436 & 600k€,30,Entre 35 & 47 m²
2,0,Vente,433000.0,3,75115,1,3P,35.40,50.0,0.0,...,0,0,1,2021-11-12,2021,11,75115_2521_00008_19,Entre 320 & 436k€ €,34,Entre 35 & 47 m²
3,1,Vente,561076.0,1,75118,0,2P,46.02,29.0,0.0,...,0,1,2,2021-01-08,2021,1,75118_2113_00110_19,Entre 436 & 600k€,50,Entre 35 & 47 m²
4,1,Vente,375980.0,1,75118,0,2P,45.76,29.0,0.0,...,0,1,2,2016-09-09,2016,9,75118_2113_00110_19,Entre 320 & 436k€ €,50,Entre 35 & 47 m²


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102577 entries, 0 to 102576
Data columns (total 41 columns):
 #   Column                                       Non-Null Count   Dtype  
---  ------                                       --------------   -----  
 0   Revente >1                                   102577 non-null  int64  
 1   libnatmut                                    102577 non-null  object 
 2   valeurfonc                                   102577 non-null  float64
 3   nblot                                        102577 non-null  object 
 4   l_codinsee                                   102577 non-null  int64  
 5   nblocdep                                     102577 non-null  int64  
 6   Type appart                                  102577 non-null  object 
 7   scarrez                                      102577 non-null  float64
 8   anarnc202012_nb_log                          102577 non-null  float64
 9   anarnc202012_nb_lot_garpark                  102577 non-nul

In [6]:
#suppression des variables qui vont gêner la modélisation :

to_drop=(["datemut",
         "moismut",
         "etaban_lot",
         "cat_valeur",
         "cat_scarrez"])
df=df.drop(to_drop, axis=1)


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102577 entries, 0 to 102576
Data columns (total 36 columns):
 #   Column                                       Non-Null Count   Dtype  
---  ------                                       --------------   -----  
 0   Revente >1                                   102577 non-null  int64  
 1   libnatmut                                    102577 non-null  object 
 2   valeurfonc                                   102577 non-null  float64
 3   nblot                                        102577 non-null  object 
 4   l_codinsee                                   102577 non-null  int64  
 5   nblocdep                                     102577 non-null  int64  
 6   Type appart                                  102577 non-null  object 
 7   scarrez                                      102577 non-null  float64
 8   anarnc202012_nb_log                          102577 non-null  float64
 9   anarnc202012_nb_lot_garpark                  102577 non-nul

In [8]:
df.isna().sum()

Revente >1                                     0
libnatmut                                      0
valeurfonc                                     0
nblot                                          0
l_codinsee                                     0
nblocdep                                       0
Type appart                                    0
scarrez                                        0
anarnc202012_nb_log                            0
anarnc202012_nb_lot_garpark                    0
anarnc202012_nb_lot_tertiaire                  0
anarnc202012_nb_lot_tot                        0
adedpe202006_logtype_baie_type_vitrage         0
adedpe202006_logtype_ch_gen_lib_princ          0
adedpe202006_logtype_classe_conso_ener         0
adedpe202006_logtype_classe_estim_ges          0
adedpe202006_logtype_traversant                0
adedpe202006_logtype_ecs_type_ener             0
adedpe202006_logtype_ph_pos_isol               0
adedpe202006_logtype_presence_climatisation    0
adedpe202006_logtype

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
# On va garder "Revente>1" comme variable cible. Suppression de "qty_of_mut"
df=df.drop("qty_of_mut", axis=1)

In [11]:
#Features / Target

X=df.drop("Revente >1", axis=1)
y=df["Revente >1"]

X.shape, y.shape

((102577, 34), (102577,))

In [12]:
numeric_columns=X.select_dtypes(include=np.number).columns.tolist()
numeric_columns

['valeurfonc',
 'l_codinsee',
 'nblocdep',
 'scarrez',
 'anarnc202012_nb_log',
 'anarnc202012_nb_lot_garpark',
 'anarnc202012_nb_lot_tertiaire',
 'anarnc202012_nb_lot_tot',
 'adedpe202006_logtype_presence_climatisation',
 'ancqpv201410_is_qpv',
 'cerffo2020_nb_log',
 'igntop202103_bat_hauteur',
 'mcumer202007_is_mer',
 'baie_orientation_sud',
 'baie_orientation_nord',
 'baie_orientation_ouest',
 'baie_orientation_est',
 'baie_orientation_autre',
 'Commerces_bruyants',
 'anneemut',
 'surface_appart']

In [13]:
object_columns=X.select_dtypes(include='object').columns.tolist()
object_columns

['libnatmut',
 'nblot',
 'Type appart',
 'adedpe202006_logtype_baie_type_vitrage',
 'adedpe202006_logtype_ch_gen_lib_princ',
 'adedpe202006_logtype_classe_conso_ener',
 'adedpe202006_logtype_classe_estim_ges',
 'adedpe202006_logtype_traversant',
 'adedpe202006_logtype_ecs_type_ener',
 'adedpe202006_logtype_ph_pos_isol',
 'adedpe202006_logtype_type_batiment',
 'cerffo2020_annee_construction',
 'cerffo2020_usage_niveau_1_txt']

In [14]:
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.25, random_state=10)
X_train.shape, X_test.shape

((76932, 34), (25645, 34))

In [15]:
#DICHOTOMISATION

In [16]:
X_train=pd.get_dummies(X_train)

In [17]:
X_test=pd.get_dummies(X_test)

In [18]:
X_train.shape

(76932, 111)

In [19]:
X_test.shape

(25645, 111)

In [20]:
# NORMALISATION

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train[numeric_columns] = scaler.fit_transform(X_train[numeric_columns])
X_test[numeric_columns] = scaler.transform(X_test[numeric_columns])

In [21]:
# Sélection des variables par Méthode Wrapper : RFE : Recursive Feature Elimination
# Le RFE consiste à tester toutes sortes de combinaisons de features et de considérer le sous-ensemble 
# qui donne les meilleurs résultats selon une métrique choisie. C'est le principe des Wrapper methods, en particulier de la RFE 
# (acronyme pour Recursive Feature Elimination).
# Le principe est simple : on fournit un modèle à l'algorithme RFE, qui est ajusté sur le jeu de données complet. 
# L'importance de chaque feature est estimée et on supprime la ou les features les moins importantes, puis on recommence. 
# Une fois le nombre de features cible atteint, on retourne le jeu de features qui a fourni le meilleur résultat 
# sur le jeu d'entrainement.

In [22]:
from sklearn.tree import DecisionTreeClassifier

dtc=DecisionTreeClassifier()

In [23]:
from sklearn.feature_selection import RFE, RFECV

selector= RFE(estimator=dtc, n_features_to_select=50)

In [24]:
selector.fit(X_train, y_train)

RFE(estimator=DecisionTreeClassifier(), n_features_to_select=50)

In [25]:
#Features sélectionnées
selector.get_support()

array([ True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True, False, False,  True,  True,  True, False,
       False, False, False, False, False, False,  True,  True, False,
        True,  True, False, False, False, False, False, False, False,
       False, False, False, False,  True,  True, False,  True, False,
       False, False, False, False, False, False,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True, False, False, False, False, False,
        True, False, False, False,  True, False,  True, False, False,
       False,  True,  True, False,  True, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False])

In [26]:
H=np.array(X_train.columns)[selector.get_support()]

In [27]:
H

array(['valeurfonc', 'l_codinsee', 'nblocdep', 'scarrez',
       'anarnc202012_nb_log', 'anarnc202012_nb_lot_garpark',
       'anarnc202012_nb_lot_tertiaire', 'anarnc202012_nb_lot_tot',
       'ancqpv201410_is_qpv', 'cerffo2020_nb_log',
       'igntop202103_bat_hauteur', 'mcumer202007_is_mer',
       'baie_orientation_sud', 'baie_orientation_nord',
       'baie_orientation_ouest', 'baie_orientation_est',
       'Commerces_bruyants', 'anneemut', 'surface_appart', 'nblot_1',
       'nblot_2', 'nblot_3', 'Type appart_2P', 'Type appart_3P',
       'adedpe202006_logtype_baie_type_vitrage_double vitrage',
       'adedpe202006_logtype_baie_type_vitrage_simple vitrage',
       'adedpe202006_logtype_ch_gen_lib_princ_chaudiere gaz condensation',
       'adedpe202006_logtype_ch_gen_lib_princ_chaudiere gaz standard',
       'adedpe202006_logtype_ch_gen_lib_princ_generateurs a effet joule',
       'adedpe202006_logtype_classe_conso_ener_C',
       'adedpe202006_logtype_classe_conso_ener_D',
       

In [28]:
#variables conservées
pd.DataFrame(H)[0].head(40)

0                                            valeurfonc
1                                            l_codinsee
2                                              nblocdep
3                                               scarrez
4                                   anarnc202012_nb_log
5                           anarnc202012_nb_lot_garpark
6                         anarnc202012_nb_lot_tertiaire
7                               anarnc202012_nb_lot_tot
8                                   ancqpv201410_is_qpv
9                                     cerffo2020_nb_log
10                             igntop202103_bat_hauteur
11                                  mcumer202007_is_mer
12                                 baie_orientation_sud
13                                baie_orientation_nord
14                               baie_orientation_ouest
15                                 baie_orientation_est
16                                   Commerces_bruyants
17                                             a

In [29]:
# Sélection des variables par SelectFromModel
# L'idée est qu'un paramètre de régularisation, noté  λ  ou  α  permet de contracter les valeurs des coefficients 
# associés à chaque variable. 
# Nous allons utiliser les coefficients renvoyés par ces algorithmes, et conserver seulement les features 
# dont les coefficients sont supérieurs à une valeur seuil.

In [30]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
selector=SelectFromModel(RandomForestClassifier(), threshold="mean")

In [31]:
selector.fit(X_train, y_train)

SelectFromModel(estimator=RandomForestClassifier(), threshold='mean')

In [32]:
#identification des variables sélectionnées
selector.get_support()

array([ True,  True,  True,  True,  True,  True,  True,  True, False,
       False,  True,  True, False, False, False, False, False, False,
        True,  True,  True, False, False,  True,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False])

In [33]:
H=np.array(X_train.columns)[selector.get_support()]

In [34]:
pd.DataFrame(H)[0].head(40)

0                                     valeurfonc
1                                     l_codinsee
2                                       nblocdep
3                                        scarrez
4                            anarnc202012_nb_log
5                    anarnc202012_nb_lot_garpark
6                  anarnc202012_nb_lot_tertiaire
7                        anarnc202012_nb_lot_tot
8                              cerffo2020_nb_log
9                       igntop202103_bat_hauteur
10                            Commerces_bruyants
11                                      anneemut
12                                surface_appart
13                                       nblot_1
14                                       nblot_2
15    adedpe202006_logtype_ph_pos_isol_non isole
Name: 0, dtype: object

In [35]:
# Sélection des variables par SELECTKBEST - score_func=f_classif
# La méthode de sélection employée ici est la sélection univariée sur la base de information mutuelle pour la régression. 
# Cette méthode élimine les variables pour lesquelles les valeurs de l’information mutuelle avec la variable de sortie 
# sont les plus faibles (c’est à dire, qui « expliquent » le moins bien la variable de sortie). 

In [36]:
from sklearn.feature_selection import SelectKBest, f_classif
selector=SelectKBest (f_classif, k=40)

In [37]:
selector.fit(X_train, y_train)

SelectKBest(k=40)

In [38]:
# Identification des variables sélectionnées
selector.get_support()

array([ True,  True,  True, False,  True,  True,  True,  True, False,
        True,  True,  True, False, False,  True, False, False, False,
       False,  True, False,  True,  True,  True,  True,  True,  True,
        True, False, False, False, False, False,  True,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False,  True, False, False, False, False, False,
        True, False, False, False,  True, False, False,  True, False,
       False, False,  True,  True, False, False,  True, False, False,
       False,  True, False, False,  True,  True, False,  True,  True,
       False, False, False,  True,  True, False, False, False, False,
       False,  True,  True, False, False, False,  True,  True, False,
       False, False, False])

In [39]:
H=np.array(X_train.columns)[selector.get_support()]

In [40]:
H

array(['valeurfonc', 'l_codinsee', 'nblocdep', 'anarnc202012_nb_log',
       'anarnc202012_nb_lot_garpark', 'anarnc202012_nb_lot_tertiaire',
       'anarnc202012_nb_lot_tot', 'ancqpv201410_is_qpv',
       'cerffo2020_nb_log', 'igntop202103_bat_hauteur',
       'baie_orientation_nord', 'anneemut', 'libnatmut_Vente',
       "libnatmut_Vente en l'état futur d'achèvement", 'nblot_1',
       'nblot_2', 'nblot_3', 'nblot_4', 'nblot_5', 'Type appart_2P',
       'Type appart_3P',
       'adedpe202006_logtype_ch_gen_lib_princ_generateurs a effet joule',
       'adedpe202006_logtype_ch_gen_lib_princ_reseau de chaleur',
       'adedpe202006_logtype_classe_conso_ener_F',
       'adedpe202006_logtype_classe_estim_ges_B',
       'adedpe202006_logtype_classe_estim_ges_E',
       'adedpe202006_logtype_traversant_Unknown',
       'adedpe202006_logtype_traversant_non traversant',
       'adedpe202006_logtype_traversant_traversant est ouest',
       'adedpe202006_logtype_traversant_traversant tout venant

In [41]:
# Voir doc Excel Sélection_var pour synthèse